In [5]:
# -*- coding: utf-8 -*-
"""
Created on Fri Sep 25 16:29:36 2020
https://gist.github.com/anonymous/d9f2948047d886bee43257b02242d1cd

@author: Usuario
"""

import pygame
import time
import random

#---------------------------------------------- DEFINICIONES GLOBALES
#

BLACK = (0,0,0)
WHITE = (255,255,255)
RED  = (255, 0, 0)

#---------------------------------------------- DEFINICIONES DE CLASE
#

class Coche(object):
    
    ancho = 73    # Ancho del coche
    x_suma = 0    # parámetros para desplazamiento
    velocidad = 0   # y velocidad

    def __init__(self, juego):
        self.x = (juego.ancho * 0.45)   # parámetros de posición
        self.y = (juego.alto * 0.8)
        self.img = pygame.image.load('.\\img\\racecar.png')

    def dibuja_y_mueve (self, juego):
        juego.ventana.fill(WHITE)                     # Blanquea la pantalla
        self.x += self.x_suma                   # recalcula la coordenada x
        juego.ventana.blit(self.img, (self.x,self.y))  # Dibuja el coche

    def acelera (self, valor):
        self.x_suma = valor
        
    def calcula_colision (self, caja) :
        if self.y < (caja.y + caja.alto) :
            if (self.x > caja.x and  self.x < (caja.x + caja.ancho)) or \
              ((self.x + self.ancho > caja.x) and (self.x + self.ancho) < (caja.x + caja.ancho)) :
                return True
        return False
     
        
class Caja(object):
    ancho = 100
    alto = 100    
    speed = 3
    
    def __init__(self, juego, color, pos_y):
        self.x = random.randrange(0, juego.ancho)
        self.y = pos_y
        self.color = color
        self.dibuja (juego)

        
    def dibuja (self, juego):   
        pygame.draw.rect(juego.ventana, self.color, [self.x, self.y,  self.ancho, self.alto])
        
    def mueve (self, juego):
        #------------------------ baja la caja
        self.y += self.speed
        
        # Si la caja sale por abajo sin choque reaparece por arriba        
        if self.y > juego.alto:                         # Si la caja se sale de la pantalla
           self.y = 0 - self.alto                       # se vuelve a situar arriba en otra 
           self.x = random.randrange(0,juego.ancho)     # coordenada aleatoria
           juego.numCaidas += 1                         # se suma 1 a las cajas caidas
           self.speed += 1                              # se aumenta la velocidad
           self.ancho += (juego.numCaidas * 1.2)        # va aumentando la anchura de la caja
           

class Juego(object):

    numCaidas = 0   # El contador de cajas será la puntuació
    ancho = 800
    alto = 600
   
    def __init__(self):
        pygame.init()
        self.ventana = pygame.display.set_mode((self.ancho, self.alto))
        pygame.display.set_caption('Pequeña carrera') 
        self.ventana.fill(WHITE)      # Blanquea la pantalla

    def play(self):

        clock = pygame.time.Clock()         # Activa el reloj
        coche = Coche(self)                 # Crea un objeto de tipo coche
        caja = Caja(self, BLACK, -600)                 # Crea un objeto de tipo caja
        caja2 = Caja(self, RED, -400)
        
        acabado = False
        #------------------------------------------ bucle del juego
        while acabado == False:
            #------------------------------------------ bucle evento
            for event in pygame.event.get():
                
                if event.type == pygame.QUIT:
                    acabado = True
                    break

                if event.type == pygame.KEYDOWN:            # Pulsan una tecla
                    if event.key == pygame.K_LEFT:
                        coche.acelera(-5)
                    elif event.key == pygame.K_RIGHT:
                        coche.acelera(5)
                        
                if event.type == pygame.KEYUP:              # Sueltan una tecla
                    if event.key == pygame.K_LEFT or \
                       event.key == pygame.K_RIGHT:
                        coche.acelera(0)
                        
            #------------------------------------------ fin bucle evento
            #

            coche.dibuja_y_mueve(self)
            
            #--------------------------------------------
            #  Dibuja la caja que cae y recalcula su altura
            
            caja.dibuja (self)
            caja.mueve (self)
            caja2.dibuja (self)
            caja2.mueve (self)
            acabado = coche.calcula_colision(caja)or coche.calcula_colision(caja2)
                
            if acabado == False:
                #------------------------------------ Calcula si coche sale por los margenes pantalla        
                if coche.x > (self.ancho - coche.ancho) or coche.x < 0:
                    acabado = True
                    
            if acabado == True:          
                self.mensaje('CRASHH !!!')
               
            self.muestra_caidas(self.numCaidas)
            pygame.display.update()
            clock.tick(60)
        #------------------------------------------ fin bucle del juego
        pygame.quit()
        time.sleep(2) 

        
    def muestra_caidas(self,count):
        font = pygame.font.SysFont(None, 25)
        text = font.render("Caidos: "+str(count), True, BLACK)
        self.ventana.blit(text,(0,0))

    def text_objects(self,text, font, color):
        textSurface = font.render(text, True, color)
        return textSurface, textSurface.get_rect()
       
    def mensaje(self, text):
        largeText = pygame.font.Font('.\\fonts\\freesansbold.ttf',80)
        TextSurf, TextRect = self.text_objects(text, largeText, BLACK)
        TextRect.center = ((self.ancho/2),(self.alto/2))
        self.ventana.blit(TextSurf, TextRect)
        pygame.display.update()
        time.sleep(2)

#---------------------------------------------- fin clase : Juego

#---------------------------------------------- INICIO DEL PROGRAMA
#

if __name__ == "__main__":
    Juego().play()
    #quit()